In [1]:
import json
from http import HTTPStatus
from typing import List, Optional, Any
from pydantic import BaseModel

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.error import HTTPError

In [15]:

class Price(BaseModel):
    mainValue: int | None = None
    emptyValue: bool | None = None
    belowPrice: bool | None = None
    multiplePrices: bool | None = None

class Address(BaseModel):
    city: str | None = None
    stateAcronym: str | None = None
    neighborhood: str | None = None
    isApproximateLocation: bool | None = None

class Image(BaseModel):
    src: str | None = None
    alt: str | None = None
    isPriority: bool | None = None

class Amenities(BaseModel):
    usableAreas: str | None = None
    bedrooms: str | None = None
    bathrooms: str | None = None
    parkingSpaces: str | None = None
    values: List[str] | None = None

class RealEstate(BaseModel):
    id: str | None = None
    legacyId: int | None = None
    name: str | None = None
    advertiserUrl: str | None = None
    tier: str | None = None
    license: str | None = None
    createdDate: str | None = None
    phoneNumbers: List[str] | None = None
    whatsAppNumber: str | None = None
    defaultMessage: str | None = None
    totalCountByFilter: int | None = None
    totalCountByAdvertiser: int | None = None
    isVerified: bool | None = None
    isPremium: bool | None = None
    imageUrl: str | None = None

class AdvertiserLogo(BaseModel):
    src: str | None = None
    alt: str | None = None

class RealEstateElement(BaseModel):
    id: str
    externalId: str
    contractType: str | None = None
    href: str | None = None
    prices: Price | None = None
    address: Address | None = None
    business: str | None = None
    highlight: str | None = None
    imageList: List[Image] | None = None
    amenities: Amenities | None = None
    realEstate: RealEstate | None = None
    visualized: bool | None = None
    description: str | None = None
    isNoWarrantorRent: bool | None = None
    constructionStatus: str | None = None
    expansionType: str | None = None
    sourceId: str | None = None
    stamps: List[str] | None = None
    unitTypes: List[str] | None = None
    displayAddressType: str | None = None
    advertiserLogo: AdvertiserLogo | None = None


In [3]:
USER_AGENT = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"

ACTION = "venda"
LOCALIZATION = "mg+pocos-de-caldas"
TYPE = "imoveis"
PAGE = 2

In [4]:
import time
from dataclasses import dataclass

import backoff
from loguru import logger


@dataclass
class RequestedPageResponse:
    html: Any
    code: int = 200
    exception: Exception | None = None

def get_page(url: str, timeout: int = 20, verbose: int = 0):

    request = Request(url)

    request.add_header("User-Agent", USER_AGENT)

    try:
        return RequestedPageResponse(html=urlopen(request, timeout=timeout))
    except HTTPError as e:
        logger.error("[error]", e)
        return RequestedPageResponse(html=None, code=e.getcode(), exception=e)



def backoff_hdlr(details):
    time.sleep(3)
    logger.warning("Backing off {wait:0.1f} seconds after {tries} tries "
           "calling function {target} with args {args} and kwargs "
           "{kwargs}".format(**details))
    
@backoff.on_exception(
    backoff.expo,
    HTTPError,
    max_tries=3,
    logger=logger,
    on_backoff=backoff_hdlr,
)
def get_page_html(page, action, type, localization):
    url = f"https://www.zapimoveis.com.br/{action}/{type}/{localization}/?pagina={page}"
    logger.debug(f"Requesting info from '{url}'")

    response = get_page(url)

    if response.code != HTTPStatus.OK:
        raise response.exception

    return response.html

def get_real_state_data(page_html):
    soup = BeautifulSoup(page_html, "html.parser")
    script = soup.find('script', id='__NEXT_DATA__')
    listings = json.loads(script.text)
    raw_data = (
        listings
        .get("props", {})
        .get("pageProps", {})
        .get("initialProps", {})
        .get("data", {})
    )
    return [RealEstateElement(**d) for d in raw_data]


In [5]:
# Como o loop deve ficar na funcao principal
for page in [1,2, 1000, 3]:
    try:
        page_html = get_page_html(page=page, action=ACTION, type=TYPE, localization=LOCALIZATION)
    except HTTPError:
        break

2024-11-21 19:39:50.757 | DEBUG    | __main__:get_page_html:43 - Requesting info from 'https://www.zapimoveis.com.br/venda/imoveis/mg+pocos-de-caldas/?pagina=1'
2024-11-21 19:39:53.484 | DEBUG    | __main__:get_page_html:43 - Requesting info from 'https://www.zapimoveis.com.br/venda/imoveis/mg+pocos-de-caldas/?pagina=1'
2024-11-21 19:39:55.256 | DEBUG    | __main__:get_page_html:43 - Requesting info from 'https://www.zapimoveis.com.br/venda/imoveis/mg+pocos-de-caldas/?pagina=1'
2024-11-21 19:39:57.304 | DEBUG    | __main__:get_page_html:43 - Requesting info from 'https://www.zapimoveis.com.br/venda/imoveis/mg+pocos-de-caldas/?pagina=1'


In [6]:
page_html = get_page_html(page=1, action=ACTION, type=TYPE, localization=LOCALIZATION)

data = get_real_state_data(page_html)

2024-11-22 01:57:36.228 | DEBUG    | __main__:get_page_html:43 - Requesting info from 'https://www.zapimoveis.com.br/venda/imoveis/mg+pocos-de-caldas/?pagina=1'


### Create Table

```sql
CREATE TABLE real_estate (
    id VARCHAR PRIMARY KEY, 
    estate_id VARCHAR,
    search_date TIMESTAMP,
    external_id VARCHAR,
    contract_type VARCHAR,
    href VARCHAR,
    
    -- Campos do modelo Price (prefixados com 'price_')
    price_main_value INTEGER,
    price_empty_value BOOLEAN,
    price_below_price BOOLEAN,
    price_multiple_prices BOOLEAN,
    
    -- Campos do modelo Address (prefixados com 'address_')
    address_city VARCHAR,
    address_state_acronym VARCHAR,
    address_neighborhood VARCHAR,
    address_is_approximate_location BOOLEAN,
    
    business VARCHAR,
    highlight VARCHAR,
    
    -- Lista de imagens armazenada como JSONB
    image_list JSONB,
    
    -- Campos do modelo Amenities (prefixados com 'amenities_')
    amenities_usable_areas VARCHAR,
    amenities_bedrooms VARCHAR,
    amenities_bathrooms VARCHAR,
    amenities_parking_spaces VARCHAR,
    amenities_values JSONB, -- Lista de valores armazenada como JSONB
    
    -- Campos do modelo RealEstate (prefixados com 'real_estate_')
    real_estate_id VARCHAR,
    real_estate_legacy_id INTEGER,
    real_estate_name VARCHAR,
    real_estate_advertiser_url VARCHAR,
    real_estate_tier VARCHAR,
    real_estate_license VARCHAR,
    real_estate_created_date TIMESTAMP,
    real_estate_phone_numbers JSONB, -- Lista de números de telefone armazenada como JSONB
    real_estate_whats_app_number VARCHAR,
    real_estate_default_message TEXT,
    real_estate_total_count_by_filter INTEGER,
    real_estate_total_count_by_advertiser INTEGER,
    real_estate_is_verified BOOLEAN,
    real_estate_is_premium BOOLEAN,
    real_estate_image_url VARCHAR,
    
    visualized BOOLEAN,
    description TEXT,
    is_no_warrantor_rent BOOLEAN,
    construction_status VARCHAR,
    expansion_type VARCHAR,
    source_id VARCHAR,
    
    -- Listas armazenadas como JSONB
    stamps JSONB,
    unit_types JSONB,
    
    display_address_type VARCHAR,
    
    -- Campos do modelo AdvertiserLogo (prefixados com 'advertiser_logo_')
    advertiser_logo_src VARCHAR,
    advertiser_logo_alt VARCHAR
);


```

In [54]:
import uuid
import pandas as pd
import uuid
from sqlalchemy import create_engine
import json
from datetime import datetime, timezone

# Supondo que você tenha os modelos Pydantic definidos conforme seu código inicial
from pydantic import BaseModel
from typing import List, Optional

def datetime_to_iso8601_z(dt):
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    else:
        dt = dt.astimezone(timezone.utc)
    return dt.strftime('%Y-%m-%dT%H:%M:%SZ')

def real_estate_element_to_dict(real_estate_element: RealEstateElement, search_date: datetime):
    formatted_search_date = datetime_to_iso8601_z(search_date)
    
    data = {
        'id': str(uuid.uuid4()),
        'estate_id': real_estate_element.id,
        'search_date': formatted_search_date,
        'external_id': real_estate_element.externalId,
        'contract_type': real_estate_element.contractType,
        'href': real_estate_element.href,

        # Campos do modelo Price
        'price_main_value': real_estate_element.prices.mainValue if real_estate_element.prices else None,
        'price_empty_value': real_estate_element.prices.emptyValue if real_estate_element.prices else None,
        'price_below_price': real_estate_element.prices.belowPrice if real_estate_element.prices else None,
        'price_multiple_prices': real_estate_element.prices.multiplePrices if real_estate_element.prices else None,

        # Campos do modelo Address
        'address_city': real_estate_element.address.city if real_estate_element.address else None,
        'address_state_acronym': real_estate_element.address.stateAcronym if real_estate_element.address else None,
        'address_neighborhood': real_estate_element.address.neighborhood if real_estate_element.address else None,
        'address_is_approximate_location': real_estate_element.address.isApproximateLocation if real_estate_element.address else None,
        'business': real_estate_element.business,
        'highlight': real_estate_element.highlight,

        # image_list como JSON
        'image_list': json.dumps([image.model_dump() for image in real_estate_element.imageList]) if real_estate_element.imageList else None,

        # Campos do modelo Amenities
        'amenities_usable_areas': real_estate_element.amenities.usableAreas if real_estate_element.amenities else None,
        'amenities_bedrooms': real_estate_element.amenities.bedrooms if real_estate_element.amenities else None,
        'amenities_bathrooms': real_estate_element.amenities.bathrooms if real_estate_element.amenities else None,
        'amenities_parking_spaces': real_estate_element.amenities.parkingSpaces if real_estate_element.amenities else None,
        'amenities_values': json.dumps(real_estate_element.amenities.values) if real_estate_element.amenities and real_estate_element.amenities.values else None,
        
        # Campos do modelo RealEstate
        'real_estate_id': real_estate_element.realEstate.id if real_estate_element.realEstate else None,
        'real_estate_legacy_id': real_estate_element.realEstate.legacyId if real_estate_element.realEstate else None,
        'real_estate_name': real_estate_element.realEstate.name if real_estate_element.realEstate else None,
        'real_estate_advertiser_url': real_estate_element.realEstate.advertiserUrl if real_estate_element.realEstate else None,
        'real_estate_tier': real_estate_element.realEstate.tier if real_estate_element.realEstate else None,
        'real_estate_license': real_estate_element.realEstate.license if real_estate_element.realEstate else None,
        'real_estate_created_date': real_estate_element.realEstate.createdDate if real_estate_element.realEstate else None,
        'real_estate_phone_numbers': json.dumps(real_estate_element.realEstate.phoneNumbers) if real_estate_element.realEstate and real_estate_element.realEstate.phoneNumbers else None,
        'real_estate_whats_app_number': real_estate_element.realEstate.whatsAppNumber if real_estate_element.realEstate else None,
        'real_estate_default_message': real_estate_element.realEstate.defaultMessage if real_estate_element.realEstate else None,
        'real_estate_total_count_by_filter': real_estate_element.realEstate.totalCountByFilter if real_estate_element.realEstate else None,
        'real_estate_total_count_by_advertiser': real_estate_element.realEstate.totalCountByAdvertiser if real_estate_element.realEstate else None,
        'real_estate_is_verified': real_estate_element.realEstate.isVerified if real_estate_element.realEstate else None,
        'real_estate_is_premium': real_estate_element.realEstate.isPremium if real_estate_element.realEstate else None,
        'real_estate_image_url': real_estate_element.realEstate.imageUrl if real_estate_element.realEstate else None,
        
        'visualized': real_estate_element.visualized,
        'description': real_estate_element.description,
        'is_no_warrantor_rent': real_estate_element.isNoWarrantorRent,
        'construction_status': real_estate_element.constructionStatus,
        'expansion_type': real_estate_element.expansionType,
        'source_id': real_estate_element.sourceId,
        'stamps': json.dumps(real_estate_element.stamps) if real_estate_element.stamps else None,
        'unit_types': json.dumps(real_estate_element.unitTypes) if real_estate_element.unitTypes else None,
        'display_address_type': real_estate_element.displayAddressType,

        # Campos do modelo AdvertiserLogo
        'advertiser_logo_src': real_estate_element.advertiserLogo.src if real_estate_element.advertiserLogo else None,
        'advertiser_logo_alt': real_estate_element.advertiserLogo.alt if real_estate_element.advertiserLogo else None,
    }

    # Converter 'real_estate_created_date' para datetime, se necessário
    if data['real_estate_created_date']:
        try:
            data['real_estate_created_date'] = datetime.fromisoformat(data['real_estate_created_date'].replace('Z', '+00:00'))
        except ValueError:
            data['real_estate_created_date'] = None

    return data


# # Exemplo de uso:

# # Suponha que 'real_estate_element' seja sua instância de RealEstateElement
# # Você pode definir sua instância aqui (omitido para brevidade)

# # Converte a instância em um dicionário
# data_dict = real_estate_element_to_dict(real_estate_element)

# # Cria um DataFrame
# df = pd.DataFrame([data_dict])

# # Detalhes de conexão com o banco de dados
# db_user = 'seu_usuario'
# db_password = 'sua_senha'
# db_host = 'seu_host'
# db_port = 'sua_porta'
# db_name = 'seu_banco'

# # Cria uma engine SQLAlchemy
# engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# # Ingestão do DataFrame no banco de dados
# df.to_sql('RealEstateElement', engine, if_exists='append', index=False)


In [55]:
search_date = datetime.now()

new_df = pd.DataFrame([real_estate_element_to_dict(x, search_date) for x in data])

In [57]:
new_df

,id,estate_id,search_date,external_id,contract_type,href,price_main_value,price_empty_value,price_below_price,price_multiple_prices,...,description,is_no_warrantor_rent,construction_status,expansion_type,source_id,stamps,unit_types,display_address_type,advertiser_logo_src,advertiser_logo_alt
0,bb367e9b-9e3f-487c-ac03-0f73e1121f4e,2750305705,2024-11-22T02:56:13Z,V98723,REAL_ESTATE,https://www.zapimoveis.com.br/imovel/venda-cas...,1800000,False,False,False,...,"Imóvel em condomínio fechado na zona Oeste, re...",False,,SUPER PREMIUM,f1fa6ce4-e68c-3a85-b3b1-805a9835ef1b,"[""FESTIVAL_DA_MUDANCA_OUTUBRO_2024""]","[""CONDOMINIUM""]",NEIGHBORHOOD,https://resizedimgs.zapimoveis.com.br/fit-in/1...,Logo do Anunciante
1,17402dbc-cd0f-4656-8d08-7b8fad29f62f,2753013316,2024-11-22T02:56:13Z,Residencial Ópera,PROPERTY_DEVELOPER,https://www.zapimoveis.com.br/lancamento/venda...,499250,False,False,True,...,**🌟 Conheça o ÓPERA: Seu Novo Lar em Poços de ...,False,Na planta,FIXED DEVELOPMENTS POSITIONS,cf2b5028-5662-3b3a-86f1-7b92c7444eae,None,"[""APARTMENT""]",ALL,https://resizedimgs.zapimoveis.com.br/fit-in/1...,Logo do Anunciante
2,2c8f083d-f884-429b-8f97-1780e9fa8ead,2748858089,2024-11-22T02:56:13Z,V83081,REAL_ESTATE,https://www.zapimoveis.com.br/imovel/venda-cas...,950000,False,False,False,...,"Imóvel todo plano em bairro nobre com sala, co...",False,,SUPER PREMIUM,115c1bed-5a09-3ace-90c0-acafdb0cc626,"[""FESTIVAL_DA_MUDANCA_OUTUBRO_2024""]","[""HOME""]",NEIGHBORHOOD,https://resizedimgs.zapimoveis.com.br/fit-in/1...,Logo do Anunciante
3,e682a6ce-b192-47b8-8f91-48cc7147f16e,2676110839,2024-11-22T02:56:13Z,V40103,REAL_ESTATE,https://www.zapimoveis.com.br/imovel/venda-cas...,420000,False,False,False,...,"Casa em dois pavimentos com 03 dormitórios, co...",False,,SUPER PREMIUM,84696afd-9e4d-3b10-ae3a-f9874656fc9d,None,"[""HOME""]",NEIGHBORHOOD,https://resizedimgs.zapimoveis.com.br/fit-in/1...,Logo do Anunciante
4,ea731a3b-9a8d-4901-b74a-51b8a8f3ece6,2757887919,2024-11-22T02:56:13Z,753352,REAL_ESTATE,https://www.zapimoveis.com.br/imovel/venda-cas...,230000,False,False,False,...,Linda e aconchegante casa térrea com quintal n...,False,,PREMIUM,516ba475-c69f-3ba5-b392-c1fbf43726c4,None,"[""HOME""]",NEIGHBORHOOD,https://resizedimgs.zapimoveis.com.br/fit-in/1...,Logo do Anunciante
5,cc55a6c2-4cec-48e6-9a86-e896b4ac32be,2753392398,2024-11-22T02:56:13Z,715794,REAL_ESTATE,https://www.zapimoveis.com.br/imovel/venda-ter...,159900,False,False,False,...,Excelente terreno a venda em ótima localização...,False,,STANDARD,0c6628cf-d836-3331-ab61-5f1a7f1d96b5,None,"[""RESIDENTIAL_ALLOTMENT_LAND""]",NEIGHBORHOOD,https://resizedimgs.zapimoveis.com.br/fit-in/1...,Logo do Anunciante
6,97705909-5d26-4178-86ed-955566d3caa3,2756849402,2024-11-22T02:56:13Z,V90923,REAL_ESTATE,https://www.zapimoveis.com.br/imovel/venda-cas...,750000,False,False,False,...,"Localizado na região das Flores, com proximida...",False,,PREMIUM,ff64bb4a-1efc-39f7-964a-3a02d42457f6,None,"[""HOME""]",NEIGHBORHOOD,https://resizedimgs.zapimoveis.com.br/fit-in/1...,Logo do Anunciante
7,839b492d-f4f4-46e3-8290-b0df31a5dac4,2539639393,2024-11-22T02:56:13Z,FOOPQ5CZ,OWNER,https://www.zapimoveis.com.br/imovel/venda-apa...,225000,False,False,False,...,RESIDÊNCIAL DONA ROSA:\n\nConheça um projeto i...,False,,PREMIUM,18a1bb4e-33b2-316e-a620-ad2bfcc7b31c,None,"[""APARTMENT""]",ALL,None,None
8,e9134048-1fec-47fa-8d54-dfcdc5de4f7a,2757330923,2024-11-22T02:56:13Z,309662X,REAL_ESTATE,https://www.zapimoveis.com.br/imovel/venda-apa...,419360,False,False,False,...,Apartamento no Residencial Paraguai a venda em...,False,,STANDARD,b1c93bf9-5343-39e1-8efb-65f3a880b36b,None,"[""APARTMENT""]",ALL,https://resizedimgs.zapimoveis.com.br/fit-in/1...,Logo do Anunciante
9,70f3ef81-546d-4e1d-a56c-509310671b64,2734079864,2024-11-22T02:56:13Z,V25652,REAL_ESTATE,https://www.zapimoveis.com.br/imovel/venda-apa...,460000,False,False,False,...,Apartamento em excelente localização fino acab...,False,,PREMIUM,26aa3d64-3eba-329b-99db-a6c1c68

In [58]:
from sqlalchemy import create_engine
import os
import pandas as pd
from urllib.parse import quote_plus

db_params = dict(
    user=os.getenv("DB_USERNAME"),
    password=quote_plus(os.getenv("DB_PASSWORD")),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    dbname=os.getenv("DB_NAME"),
)

db_url = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

try:
    engine = create_engine(db_url)
    new_df.to_sql("real_estate", engine, if_exists='append', index=False)
    
except Exception as e:
    print(f"Error creating SQLAlchemy engine: {e}")
    raise


In [50]:
new_df

,id,estateId,seaarchDate,externalId,contractType,href,price_mainValue,price_emptyValue,price_belowPrice,price_multiplePrices,...,description,isNoWarrantorRent,constructionStatus,expansionType,sourceId,stamps,unitTypes,displayAddressType,advertiserLogo_src,advertiserLogo_alt
0,11b1688a-1c6a-4c71-827b-96f74833e9eb,2750305705,2024-11-22T02:40:00Z,V98723,REAL_ESTATE,https://www.zapimoveis.com.br/imovel/venda-cas...,1800000,False,False,False,...,"Imóvel em condomínio fechado na zona Oeste, re...",False,,SUPER PREMIUM,f1fa6ce4-e68c-3a85-b3b1-805a9835ef1b,"[""FESTIVAL_DA_MUDANCA_OUTUBRO_2024""]","[""CONDOMINIUM""]",NEIGHBORHOOD,https://resizedimgs.zapimoveis.com.br/fit-in/1...,Logo do Anunciante
1,d8ce2b80-6d00-4daf-97b1-351e92a92923,2753013316,2024-11-22T02:40:00Z,Residencial Ópera,PROPERTY_DEVELOPER,https://www.zapimoveis.com.br/lancamento/venda...,499250,False,False,True,...,**🌟 Conheça o ÓPERA: Seu Novo Lar em Poços de ...,False,Na planta,FIXED DEVELOPMENTS POSITIONS,cf2b5028-5662-3b3a-86f1-7b92c7444eae,None,"[""APARTMENT""]",ALL,https://resizedimgs.zapimoveis.com.br/fit-in/1...,Logo do Anunciante
2,8fd056da-b049-43fb-8a08-afbc814103a5,2748858089,2024-11-22T02:40:00Z,V83081,REAL_ESTATE,https://www.zapimoveis.com.br/imovel/venda-cas...,950000,False,False,False,...,"Imóvel todo plano em bairro nobre com sala, co...",False,,SUPER PREMIUM,115c1bed-5a09-3ace-90c0-acafdb0cc626,"[""FESTIVAL_DA_MUDANCA_OUTUBRO_2024""]","[""HOME""]",NEIGHBORHOOD,https://resizedimgs.zapimoveis.com.br/fit-in/1...,Logo do Anunciante
3,3c0bc0d2-a4bf-40a8-916e-62c9ac2c2325,2676110839,2024-11-22T02:40:00Z,V40103,REAL_ESTATE,https://www.zapimoveis.com.br/imovel/venda-cas...,420000,False,False,False,...,"Casa em dois pavimentos com 03 dormitórios, co...",False,,SUPER PREMIUM,84696afd-9e4d-3b10-ae3a-f9874656fc9d,None,"[""HOME""]",NEIGHBORHOOD,https://resizedimgs.zapimoveis.com.br/fit-in/1...,Logo do Anunciante
4,545d36e8-0166-4e2c-ac62-edeb32be6641,2757887919,2024-11-22T02:40:00Z,753352,REAL_ESTATE,https://www.zapimoveis.com.br/imovel/venda-cas...,230000,False,False,False,...,Linda e aconchegante casa térrea com quintal n...,False,,PREMIUM,516ba475-c69f-3ba5-b392-c1fbf43726c4,None,"[""HOME""]",NEIGHBORHOOD,https://resizedimgs.zapimoveis.com.br/fit-in/1...,Logo do Anunciante
5,21eba86e-ed06-464a-b8e7-1336756f9642,2753392398,2024-11-22T02:40:00Z,715794,REAL_ESTATE,https://www.zapimoveis.com.br/imovel/venda-ter...,159900,False,False,False,...,Excelente terreno a venda em ótima localização...,False,,STANDARD,0c6628cf-d836-3331-ab61-5f1a7f1d96b5,None,"[""RESIDENTIAL_ALLOTMENT_LAND""]",NEIGHBORHOOD,https://resizedimgs.zapimoveis.com.br/fit-in/1...,Logo do Anunciante
6,433c7245-cbb0-4f52-b8a2-95ae80b821c7,2756849402,2024-11-22T02:40:00Z,V90923,REAL_ESTATE,https://www.zapimoveis.com.br/imovel/venda-cas...,750000,False,False,False,...,"Localizado na região das Flores, com proximida...",False,,PREMIUM,ff64bb4a-1efc-39f7-964a-3a02d42457f6,None,"[""HOME""]",NEIGHBORHOOD,https://resizedimgs.zapimoveis.com.br/fit-in/1...,Logo do Anunciante
7,7f9ee332-6419-4097-bb3f-f2a447315508,2539639393,2024-11-22T02:40:00Z,FOOPQ5CZ,OWNER,https://www.zapimoveis.com.br/imovel/venda-apa...,225000,False,False,False,...,RESIDÊNCIAL DONA ROSA:\n\nConheça um projeto i...,False,,PREMIUM,18a1bb4e-33b2-316e-a620-ad2bfcc7b31c,None,"[""APARTMENT""]",ALL,None,None
8,5bf19616-e130-435b-b81c-298d4f78a6bf,2757330923,2024-11-22T02:40:00Z,309662X,REAL_ESTATE,https://www.zapimoveis.com.br/imovel/venda-apa...,419360,False,False,False,...,Apartamento no Residencial Paraguai a venda em...,False,,STANDARD,b1c93bf9-5343-39e1-8efb-65f3a880b36b,None,"[""APARTMENT""]",ALL,https://resizedimgs.zapimoveis.com.br/fit-in/1...,Logo do Anunciante
9,03fa1786-c27e-41ce-bcf6-9e9d01fe3f02,2734079864,2024-11-22T02:40:00Z,V25652,REAL_ESTATE,https://www.zapimoveis.com.br/imovel/venda-apa...,460000,False,False,False,...,Apartamento em excelente localização fino acab...,False,,PREMIUM,26aa3d64-3eba-329b-99db-a6c1c681aae1,"[""FESTIVAL

In [23]:
data[0].imageList

[Image(src='https://resizedimgs.zapimoveis.com.br/crop/614x297/vr.images.sp/670d30eb3d610323bb4544ffcb02dbe3.webp', alt='Imagem do imóvel', isPriority=False),
 Image(src='https://resizedimgs.zapimoveis.com.br/crop/614x297/vr.images.sp/b439025da68da585598319bbff9bdb0e.webp', alt='Imagem do imóvel', isPriority=False),
 Image(src='https://resizedimgs.zapimoveis.com.br/crop/614x297/vr.images.sp/6be543afe8d3f6c51c185e0000c5206e.webp', alt='Imagem do imóvel', isPriority=False),
 Image(src='https://resizedimgs.zapimoveis.com.br/crop/614x297/vr.images.sp/dd6fec3ab8669acffe2cfb9506e55d43.webp', alt='Imagem do imóvel', isPriority=False),
 Image(src='https://resizedimgs.zapimoveis.com.br/crop/614x297/vr.images.sp/52beeca3df194455c91641d8e4d42caa.webp', alt='Imagem do imóvel', isPriority=False),
 Image(src='https://resizedimgs.zapimoveis.com.br/crop/614x297/vr.images.sp/835aebf56d6544c0f0793ba3b99b43e9.webp', alt='Imagem do imóvel', isPriority=False),
 Image(src='https://resizedimgs.zapimoveis.com

In [44]:
from sqlalchemy import create_engine
import os
import pandas as pd
from urllib.parse import quote_plus

db_params = dict(
    user=os.getenv("DB_USERNAME"),
    password=quote_plus(os.getenv("DB_PASSWORD")),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    dbname=os.getenv("DB_NAME"),
)

# Cria a URL de conexão para SQLAlchemy
db_url = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

try:
    engine = create_engine(db_url)
    print("Connection successful!")
except Exception as e:
    print(f"Error creating SQLAlchemy engine: {e}")
    raise

query = """
SELECT *
FROM pocos_de_caldas.real_estate
LIMIT 10;
"""

try:
    df = pd.read_sql_query(query, engine)
    print("Query executed successfully!")
except Exception as e:
    print(f"Error executing query: {e}")
    raise


Connection successful!
Query executed successfully!


In [45]:
df

,id,estateid,searchdate,externalid,contracttype,href,price_mainvalue,price_emptyvalue,price_belowprice,price_multipleprices,...,description,isnowarrantorrent,constructionstatus,expansiontype,sourceid,stamps,unittypes,displayaddresstype,advertiserlogo_src,advertiserlogo_alt


In [48]:
import os
import psycopg2
import pandas as pd


db_params = dict(
    user=os.getenv("DB_USERNAME"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    dbname=os.getenv("DB_NAME"),
)

try:
    conn = psycopg2.connect(**db_params)
    print("Connection successful!")
except Exception as e:
    print(f"Error connecting to the database: {e}")
    raise

query = """
SELECT table_schema, table_name, column_name, data_type
FROM information_schema.columns
WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
ORDER BY table_schema, table_name, ordinal_position;
"""

try:
    df = pd.read_sql_query(query, conn)
    print("Query executed successfully!")
except Exception as e:
    print(f"Error executing query: {e}")
    raise
finally:
    conn.close()

print(df)


Connection successful!
Query executed successfully!
       table_schema                   table_name         column_name  \
0   pocos_de_caldas  pocos_de_caldas.real_estate                  id   
1   pocos_de_caldas  pocos_de_caldas.real_estate            estateId   
2   pocos_de_caldas  pocos_de_caldas.real_estate         seaarchDate   
3   pocos_de_caldas  pocos_de_caldas.real_estate          externalId   
4   pocos_de_caldas  pocos_de_caldas.real_estate        contractType   
..              ...                          ...                 ...   
91  pocos_de_caldas                  real_estate              stamps   
92  pocos_de_caldas                  real_estate           unittypes   
93  pocos_de_caldas                  real_estate  displayaddresstype   
94  pocos_de_caldas                  real_estate  advertiserlogo_src   
95  pocos_de_caldas                  real_estate  advertiserlogo_alt   

            data_type  
0                text  
1                text  
2          

/tmp/ipykernel_11986/2720085143.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [41]:
from sqlalchemy import create_engine
import os
import pandas as pd
from urllib.parse import quote_plus

db_params = dict(
    user=os.getenv("DB_USERNAME"),
    password=quote_plus(os.getenv("DB_PASSWORD")),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    dbname=os.getenv("DB_NAME"),
)

# Cria a URL de conexão para SQLAlchemy
db_url = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

try:
    engine = create_engine(db_url)
    print("Connection successful!")
except Exception as e:
    print(f"Error creating SQLAlchemy engine: {e}")
    raise

query = """
SELECT table_schema, table_name, column_name, data_type
FROM information_schema.columns
WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
ORDER BY table_schema, table_name, ordinal_position;
"""

try:
    # Use diretamente o engine no pandas
    df = pd.read_sql_query(query, engine)
    print("Query executed successfully!")
except Exception as e:
    print(f"Error executing query: {e}")
    raise


Connection successful!
Query executed successfully!
       table_schema   table_name                        column_name  \
0   pocos_de_caldas  real_estate                                 id   
1   pocos_de_caldas  real_estate                           estateid   
2   pocos_de_caldas  real_estate                         searchdate   
3   pocos_de_caldas  real_estate                         externalid   
4   pocos_de_caldas  real_estate                       contracttype   
5   pocos_de_caldas  real_estate                               href   
6   pocos_de_caldas  real_estate                    price_mainvalue   
7   pocos_de_caldas  real_estate                   price_emptyvalue   
8   pocos_de_caldas  real_estate                   price_belowprice   
9   pocos_de_caldas  real_estate               price_multipleprices   
10  pocos_de_caldas  real_estate                       address_city   
11  pocos_de_caldas  real_estate               address_stateacronym   
12  pocos_de_caldas  real

In [15]:
df.to_parquet("imoveis_pc_v01.parquet")

In [13]:
query = """
SELECT *
FROM pocos_de_caldas.imoveis
"""

try:
    # Use diretamente o engine no pandas
    df = pd.read_sql_query(query, engine)
    print("Query executed successfully!")
except Exception as e:
    print(f"Error executing query: {e}")
    raise

print(df)

Query executed successfully!
          id                        search_id      id_zap search_date  \
0          1  2503300374__2021_06_04_00_23_37  2503300374  2021-06-04   
1          2  2462358530__2021_06_04_00_23_37  2462358530  2021-06-04   
2          3    70958901__2021_06_04_00_23_37    70958901  2021-06-04   
3          4  2500808474__2021_06_04_00_23_40  2500808474  2021-06-04   
4          5  2503298489__2021_06_04_00_23_40  2503298489  2021-06-04   
...      ...                              ...         ...         ...   
96333  99492  2600939894__2023_05_07_12_10_10  2600939894  2023-05-07   
96334  99493  2591450014__2023_05_07_12_10_10  2591450014  2023-05-07   
96335  99494  2605146978__2023_05_07_12_10_10  2605146978  2023-05-07   
96336  99495  2568096535__2023_05_07_12_10_10  2568096535  2023-05-07   
96337  99496  2600937648__2023_05_07_12_10_10  2600937648  2023-05-07   

                type  n_parking_spaces  n_bathrooms  n_bedrooms   area  \
0               HOME

In [24]:
import os
import pandas as pd
from sqlalchemy import create_engine

# Configuração da URL de conexão
DATABASE_URL = "postgresql://{user}:{password}@{host}:{port}/{database}".format(
    user=os.getenv("DB_USERNAME"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    database=os.getenv("DB_NAME"),
)

# Criando o engine
engine = create_engine(DATABASE_URL)

# Query para obter o schema
query = """
SELECT table_schema, table_name, column_name, data_type
FROM information_schema.columns
WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
ORDER BY table_schema, table_name, ordinal_position;
"""

# Conexão explícita com o engine
with engine.connect() as connection:
    df = pd.read_sql(query, con=connection)  # Use a conexão explícita aqui

# Exibir as primeiras linhas do DataFrame
print(df.head())


/tmp/ipykernel_94/782219150.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=connection)  # Use a conexão explícita aqui


AttributeError: 'Connection' object has no attribute 'cursor'

Nesse código:
```
import pandas as pd
from sqlalchemy import create_engine

# Configuração da URL de conexão
DATABASE_URL = "postgresql://{user}:{password}@{host}:{port}/{database}".format(
    user=os.getenv("DB_USERNAME"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    database=os.getenv("DB_NAME"),
)


engine = create_engine(DATABASE_URL)


query = """
SELECT table_schema, table_name, column_name, data_type
FROM information_schema.columns
WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
ORDER BY table_schema, table_name, ordinal_position;
"""

with engine.connect() as connection:
    df = pd.read_sql(query, connection)

print(df.head())
```

tive esse erro:

```
AttributeError                            Traceback (most recent call last)
Cell In[22], line 23
     14 engine = create_engine(DATABASE_URL)
     17 query = """
     18 SELECT table_schema, table_name, column_name, data_type
     19 FROM information_schema.columns
     20 WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
     21 ORDER BY table_schema, table_name, ordinal_position;
     22 """
---> 23 df = pd.read_sql(query, engine)
     25 print(df.head())

File /opt/conda/lib/python3.11/site-packages/pandas/io/sql.py:706, in read_sql(sql, con, index_col, coerce_float, params, parse_dates, columns, chunksize, dtype_backend, dtype)
    704 with pandasSQL_builder(con) as pandas_sql:
    705     if isinstance(pandas_sql, SQLiteDatabase):
--> 706         return pandas_sql.read_query(
    707             sql,
    708             index_col=index_col,
    709             params=params,
    710             coerce_float=coerce_float,
    711             parse_dates=parse_dates,
    712             chunksize=chunksize,
    713             dtype_backend=dtype_backend,
    714             dtype=dtype,
    715         )
    717     try:
    718         _is_table_name = pandas_sql.has_table(sql)

File /opt/conda/lib/python3.11/site-packages/pandas/io/sql.py:2738, in SQLiteDatabase.read_query(self, sql, index_col, coerce_float, parse_dates, params, chunksize, dtype, dtype_backend)
   2727 def read_query(
   2728     self,
   2729     sql,
   (...)
   2736     dtype_backend: DtypeBackend | Literal["numpy"] = "numpy",
   2737 ) -> DataFrame | Iterator[DataFrame]:
-> 2738     cursor = self.execute(sql, params)
   2739     columns = [col_desc[0] for col_desc in cursor.description]
   2741     if chunksize is not None:

File /opt/conda/lib/python3.11/site-packages/pandas/io/sql.py:2672, in SQLiteDatabase.execute(self, sql, params)
   2670     raise TypeError("Query must be a string unless using sqlalchemy.")
   2671 args = [] if params is None else [params]
-> 2672 cur = self.con.cursor()
   2673 try:
   2674     cur.execute(sql, *args)

AttributeError: 'Engine' object has no attribute 'cursor'
```

In [19]:
data[0]

Conexão bem-sucedida!
Versão do PostgreSQL: ('PostgreSQL 9.5.1 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.4.7 20120313 (Red Hat 4.4.7-16), 64-bit',)


In [7]:
[d.unitTypes for d in data]

[['HOME'],
 ['APARTMENT'],
 ['APARTMENT'],
 ['APARTMENT'],
 ['HOME'],
 ['RESIDENTIAL_ALLOTMENT_LAND'],
 ['APARTMENT'],
 ['APARTMENT'],
 ['HOME'],
 ['HOME'],
 ['APARTMENT'],
 ['HOME'],
 ['APARTMENT'],
 ['HOME'],
 ['PENTHOUSE']]

In [134]:
real_state_element = data[2]

In [135]:
real_state_element.unitTypes

['HOME']

In [136]:
real_state_element.prices.mainValue

480000

In [137]:
real_state_element.address.neighborhood

'Monte Verde'

In [138]:
real_state_element.amenities

Amenities(usableAreas='161', bedrooms='4', bathrooms='3', parkingSpaces='1', values=['DEPOSIT', 'BATHTUB', 'SERVICE_AREA'])

In [139]:
real_state_element.description

'CASA COM 04 DORMITÓRIOS, SENDO 01 SUÍTE, SALA, COZINHA, BANHEIRO SOCIAL, ÁREA DE SERVIÇO COM DORMITÓRIO, DESPENSA E GARAGEM.\n\nIMOVEL COM LIGAÇÃO DE AQUECEDOR SOLAR, CERCA ELÉTRICA, PORTÃO ELETRÔNICO. \n\nPARA SEU CONFORTO, POSSUÍ BANHEIRA DE HIDROMASSAGEM NA SUÍTE. NO QUINTAL, POSSUÍ UMA EDÍCULA COMPOSTA POR DORMITÓRIO E BANHEIRO.\n\nRUA DE FÁCIL ACESSO E PRÓXIMO À GRANDE VARIEDADES COMERCIAIS NA REGIÃO...\n\n- SUPERMERCADOS,\n- PADARIAS,\n- ESCOLAS,\n- FARMÁCIAS,\n- PET SHOPS,\n- RESTAURANTE,\n- PONTO DE ÔNIBUS...!!\n\nVENHA CONHECER ESTE LINDO IMÓVEL.. LIGUE E AGENDE UMA VISITA!!. ref. 58832 atualizado em 19/11.'

In [29]:
data[0]



Element(id='2696156001', externalId='V54803', contractType='REAL_ESTATE', href='https://www.zapimoveis.com.br/imovel/venda-terreno-lote-condominio-jardim-europa-pocos-de-caldas-mg-480m2-id-2696156001/', prices=Price(mainValue=575000, emptyValue=False, belowPrice=False, multiplePrices=False), address=Address(city='Poços de Caldas', stateAcronym='MG', neighborhood='Jardim Europa', isApproximateLocation=True), business='SALE', highlight='SUPER', imageList=[Image(src='https://resizedimgs.zapimoveis.com.br/crop/614x297/vr.images.sp/5daa172b250f9fb3cbf5028b3dfbe3c5.webp', alt='Imagem do imóvel', isPriority=True), Image(src='https://resizedimgs.zapimoveis.com.br/crop/614x297/vr.images.sp/f133ac4e9c94ad1d32d236db41923968.webp', alt='Imagem do imóvel', isPriority=False)], amenities=Amenities(usableAreas='480', bedrooms='0', bathrooms='0', parkingSpaces='0', values=[]), realEstate=RealEstate(id='281c0fe5-98c1-c9f9-d96c-a2e58397c49c', legacyId=3618805, name='BG & C IMOVEIS LTDA', advertiserUrl='/

In [30]:
raw_data[0]

{'id': '2696156001',
 'externalId': 'V54803',
 'contractType': 'REAL_ESTATE',
 'href': 'https://www.zapimoveis.com.br/imovel/venda-terreno-lote-condominio-jardim-europa-pocos-de-caldas-mg-480m2-id-2696156001/',
 'prices': {'mainValue': 575000,
  'emptyValue': False,
  'belowPrice': False,
  'multiplePrices': False},
 'address': {'city': 'Poços de Caldas',
  'stateAcronym': 'MG',
  'neighborhood': 'Jardim Europa',
  'isApproximateLocation': True},
 'business': 'SALE',
 'highlight': 'SUPER',
 'imageList': [{'src': 'https://resizedimgs.zapimoveis.com.br/crop/614x297/vr.images.sp/5daa172b250f9fb3cbf5028b3dfbe3c5.webp',
   'alt': 'Imagem do imóvel',
   'isPriority': True},
  {'src': 'https://resizedimgs.zapimoveis.com.br/crop/614x297/vr.images.sp/f133ac4e9c94ad1d32d236db41923968.webp',
   'alt': 'Imagem do imóvel',
   'isPriority': False}],
 'amenities': {'usableAreas': '480',
  'bedrooms': '0',
  'bathrooms': '0',
  'parkingSpaces': '0',
  'values': []},
 'realEstate': {'id': '281c0fe5-98c

In [ ]:
import aiohttp
import asyncio
import os

async def download_image(session, url, output_folder):
    """Baixar uma única imagem."""
    try:
        async with session.get(url) as response:
            if response.status == 200:
                # Nome do arquivo baseado na URL
                filename = os.path.basename(url.split("?")[0])
                filepath = os.path.join(output_folder, filename)
                # Salvar a imagem
                with open(filepath, "wb") as file:
                    file.write(await response.read())
                print(f"Imagem salva: {filepath}")
    except Exception as e:
        print(f"Erro ao baixar {url}: {e}")

async def download_all_images(image_urls, output_folder):
    """Baixar todas as imagens em paralelo."""
    os.makedirs(output_folder, exist_ok=True)  # Criar a pasta se não existir
    async with aiohttp.ClientSession() as session:
        # Criar tarefas para baixar todas as imagens
        tasks = [download_image(session, url, output_folder) for url in image_urls]
        # Executar todas as tarefas em paralelo
        await asyncio.gather(*tasks)

# Exemplo de uso
output_folder = "images"  # Pasta onde as imagens serão salvas
asyncio.run(download_all_images(image_urls, output_folder))


# Test

In [ ]:
def get_page(url: str, timeout: int = 20, verbose: int = 0):

    request = Request(url)

    request.add_header("User-Agent", USER_AGENT)

    try:
        return RequestedPageResponse(html=urlopen(request, timeout=timeout))
    except HTTPError as e:
        logger.error("[error]", e)
        return RequestedPageResponse(html=None, code=e.getcode(), exception=e)



In [9]:
import nest_asyncio
from playwright.async_api import async_playwright
import asyncio

# Permite rodar asyncio no Jupyter
nest_asyncio.apply()

async def fetch_images_with_playwright(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            viewport={"width": 1920, "height": 1080}
        )
        page = await context.new_page()
        await page.goto(url)

        # Obtém todas as URLs das imagens
        image_urls = await page.evaluate('''() => {
            return Array.from(document.querySelectorAll('img')).map(img => img.src);
        }''')

        await browser.close()
        return image_urls

url = f"https://www.zapimoveis.com.br/{ACTION}/{TYPE}/{LOCALIZATION}/?pagina={PAGE}"
images = asyncio.run(fetch_images_with_playwright(url))

# Exibe as imagens
print("Imagens encontradas:")
print("\n".join(images))

Imagens encontradas:
https://cdn-zap-ssr-prod.zapimoveis.com.br/_next/public/assets/images/zap.svg
https://resizedimgs.zapimoveis.com.br/crop/614x297/vr.images.sp/0ded15001866524a67bdaffa97f64e51.webp
https://resizedimgs.zapimoveis.com.br/crop/614x297/vr.images.sp/d1fb746c4704455ffa22ac37c6edf7d2.webp
https://resizedimgs.zapimoveis.com.br/crop/614x297/vr.images.sp/4090544f4f82c43e6f93d0195c5001bc.webp
https://resizedimgs.zapimoveis.com.br/crop/614x297/vr.images.sp/bf8e407ecd22bd8d3373cb39ea300a4d.webp
https://resizedimgs.zapimoveis.com.br/crop/614x297/vr.images.sp/e034a60d801a6f03c10ec03856db441d.webp
https://cdnfiles.vivareal.com/campanhas/2024/festival-da-mudanca-outubro/stamps/stamps-zap.svg
https://resizedimgs.zapimoveis.com.br/crop/614x297/vr.images.sp/328e1c8331130285b7f2c31de394bfcc.webp
https://resizedimgs.zapimoveis.com.br/crop/614x297/vr.images.sp/b6dc3f35bc0edf221889566474dc097c.webp
https://resizedimgs.zapimoveis.com.br/crop/614x297/vr.images.sp/be972de69ecdbd0c0eeec5b49bda